# Week 6 Tutorial :
## Notes
1. Try to get everything working in simpler steps
1. Try each of these cells in turn 
1. You may have to restart your Notebook kernel frequently

## Step 1 - Can we get a simple Leaflet map rendered?


In [ ]:
# Read through the Module Resources!  In the docs for leaflet, I found this simple example, and it's a quick 
# test to see if it works:

# We need to import the Python libraries/modules that we need
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash
import dash_leaflet as dl

# Very important step:  when building a Dash app outside of Jupyter,
# we use the Dash() class.  When building for Jupyter, we use the
# JupyterDash() class.  
app = JupyterDash()

# All Dash apps start with a "layout"
app.layout = dl.Map(dl.TileLayer(), style={'width': '1000px', 'height': '500px'})

# for a Jupyter object:
app.run_server(port=18843)

# You should see a map of the world, centered on London, England!

## Step 2 - Can we get a Dash DataTable rendered?

In [ ]:
# Now let's see if we can get a Dash DataTable to work.  Follow the same approach:  I found this example in 
# the docs for Dash DataTable:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

from dash import dash_table
import pandas as pd

# Use the Pandas module to read a CSV file
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/solar.csv')

# Create out main object
app = JupyterDash()

# Then we always define the layout:
app.layout = dash_table.DataTable(df.to_dict('records'), [{"name": i, "id": i} for i in df.columns])

# Run our Dash app:  note that specifying 'inline' means the output will go right 
# into this Notebook.  If you omit 'inline' then you should see a hyperlink that will
# open a new tab in your browser and render the Web page
app.run_server(port=18843)

# Do you see a table of values?

## Step 3 - Test my CRUD class with just Panda DataFrame

In [ ]:
# Recall that our CRUD class is in an external .py file, so
# we use sys.path.append to help Python understand where the file is
import sys, os
sys.path.append(os.path.expanduser('~/Desktop'))
from MyClassFile import AACDatabaseLayer

# Enviroment Variables
mPort = os.environ["MONGO_PORT"]
mUser = os.environ["MONGO_USER"]
mPass = os.environ["MONGO_PASS"]
mHost = os.environ["MONGO_HOST"]

# These imports are needed for Pandas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
#
# Create an object from our CRUD class:
shelter = AACDatabaseLayer(mHost, mPort, mUser,mPass)
shelter.connect('AAC')

# Set the database to use
shelter.setDatabase('AAC')

# Now create a Pandas DataFrame by calling our CRUD object's 'read' method:
df = pd.DataFrame.from_records(list(shelter.read('animals',{'name' : 'Kevin'})))
print(df.to_dict("records"))

# You should see the result of our query directly below... This just demonstrates that we
# successfully used the Panda DataFrame object

 Step 4 - Ok, now let's add a Dash DataTable to see if we can make the output neater

In [ ]:
from jupyter_dash import JupyterDash

# This is how we tell Python where our CRUD class is:
import sys, os
sys.path.append(os.path.expanduser('~/Desktop'))
from MyClassFile import AACDatabaseLayer

# Import all the Dash modules that we need
import dash
import dash_leaflet as dl
#import dash_core_components as dcc
from dash import dcc
#import dash_html_components as html
from dash import html
import plotly.express as px
#import dash_table
from dash import dash_table
from dash.dependencies import Input, Output

# Import all the modules needed for Pandas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# The BSON utilities are imported in case you need them:
from bson.json_util import dumps
from bson.json_util import loads
import json

# Create an object from our CRUD class:
shelter = AACDatabaseLayer(mHost, mPort, mUser, mPass)
shelter.connect('AAC')

# Set the database to use
shelter.setDatabase('AAC')

# Create the DataFrame (just like we did in last example)
# NOTE:  You will need to return ALL documents, not just the ones with 
# 'name' equal to 'Kevin'
df = pd.DataFrame.from_records(shelter.read('animals',{'name' : 'Kevin'}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

# Create our JupyterDash app:
app = JupyterDash('SimpleExample')

# Every Dash app needs a layout:
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    # Here's where we get brave:  can we display our DataFrame in a DataTable?
    dash_table.DataTable(
        id='datatable-id',
        data=df.to_dict('records'),
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": False} for i in df.columns
        ],
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        # Here's how I limited the number of documents displayed at a time:
        page_size=10
    ),
    html.Br(),
    html.Hr(),
])

# Run the Dash app
app.run_server(port=18843)

# Do you see a table with your animal documents displayed?

Step 5.  Now render the table in a proper Web frame:

In [ ]:
from jupyter_dash import JupyterDash

# Tell Python where our CRUD class is
import sys, os

sys.path.append(os.path.expanduser('~/Desktop'))
from MyClassFile import AACDatabaseLayer

# Import all the needed modules
import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from bson.json_util import dumps
import json
import base64

# Create an object from our CRUD class:
shelter = AACDatabaseLayer(mHost, mPort, mUser, mPass)
shelter.connect('AAC')

# Set the database to use
shelter.setDatabase('AAC')

# Here's the company logo:
image_url = 'https://learn.snhu.edu/content/enforced/1347144-CS-340-X6151-OL-TRAD-UG.23EW6/course_documents/Grazioso%20Salvare%20Logo.png?_%26d2lSessionVal=ScQoRYLN9OTGv4T9RbCyYwie6%26ou=1332057%26_&d2lSessionVal=yWL9G0trDzGdnu2AymLP5HEmi&ou=1347144'
# Make sure you return ALL documents, not just the ones for 'Kevin', like I'm
# doing here:
# df = pd.DataFrame.from_records(shelter.read('animals',{'name' : 'Kevin'}))

data = shelter.read('animals', {})
#print("Retrieved data from MongoB:", data)

#print("Dataframe:", df)

if data is not None:
    for record in data:
        if '_id' in record:
            record['_id'] = str(record['_id'])
            
        df = pd.DataFrame.from_records(data)

else:
    # Handle the case when data is Non (empty result from MongoDB)
    df = pd.DataFrame() # Empty DataFrame

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

app = JupyterDash('SimpleExample')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    
    # customer image location, can you put it in different location and resize?
    html.Img(id='customer-image',src=image_url,
             alt='customer image', style={'height':'10%', 'width':'10%'}),

    dash_table.DataTable(
        id='datatable-id',
        data=df.to_dict('records'),
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        column_selectable='multi',
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        editable=True,  # Allow editing the cells in the table
        filter_action="native",  # Enable native filtering
        sort_action="native",  # Enable native sorting
        sort_mode="multi",  # Allow multi-column sorting
        page_size=15,  # Number of rows per page
        style_table={'overflowX': 'auto'},  # Enable horizontal scrolling for wide tables
        style_cell={'textAlign': 'left'},  # Align cell text to the left
        style_data_conditional=[  # Custom styling for odd and even rows
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': 'rgb(248, 248, 248)'
            }
        ]
    ),
    html.Br(),
    html.Hr(),
])
# Add callbacks next:

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    print(selected_columns)
    if selected_columns is None:
        return []
    
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Open in new tab:
app.run_server(port=18843, debug=True)

Step 6 - Here's a hint to get the geolocation map functionality working:

In [ ]:
from jupyter_dash import JupyterDash

# Tell Python where our CRUD class is
import sys, os

sys.path.append(os.path.expanduser('~/Desktop'))
from MyClassFile import AACDatabaseLayer

# Import all the needed modules
import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from bson.json_util import dumps
import json
import base64

from IPython.display import display

# Create an object from our CRUD class:
shelter = AACDatabaseLayer(mHost, mPort, mUser, mPass)
shelter.connect('AAC')

# Set the database to use
shelter.setDatabase('AAC')

# Here's the company logo:
image_url = 'https://learn.snhu.edu/content/enforced/1347144-CS-340-X6151-OL-TRAD-UG.23EW6/course_documents/Grazioso%20Salvare%20Logo.png?_%26d2lSessionVal=ScQoRYLN9OTGv4T9RbCyYwie6%26ou=1332057%26_&d2lSessionVal=yWL9G0trDzGdnu2AymLP5HEmi&ou=1347144'

data = shelter.read('animals', {})

if data is not None:
    for record in data:
        if '_id' in record:
            record['_id'] = str(record['_id'])
            
        df = pd.DataFrame.from_records(data)

else:
    # Handle the case when data is Non (empty result from MongoDB)
    df = pd.DataFrame() # Empty DataFrame

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

app = JupyterDash('SimpleExample')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Div(
        id='map-id', 
        className='col s12 m6',
    ),
    html.Center(html.B(html.H1('Sarah Schmidt CS-340 MongoB Authentication'))),
    html.Hr(),
    
    # customer image location, can you put it in different location and resize?
    html.Img(id='customer-image',src=image_url,
             alt='customer image', style={'height':'10%', 'width':'10%'}),

    dash_table.DataTable(
        id='datatable-id',
        data=df.to_dict('records'),
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        column_selectable='multi',
        selected_rows=[0],
        row_selectable="single",
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        editable=True,  # Allow editing the cells in the table
        filter_action="native",  # Enable native filtering
        sort_action="native",  # Enable native sorting
        sort_mode="multi",  # Allow multi-column sorting
        page_size=10,  # Number of rows per page
        style_table={'overflowX': 'auto'},  # Enable horizontal scrolling for wide tables
        style_cell={'textAlign': 'left'},  # Align cell text to the left
        style_data_conditional=[  # Custom styling for odd and even rows
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': 'rgb(248, 248, 248)'
            }
        ]
    ),
    html.Br(),
    html.Hr(),
])

# Add callbacks next:

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
#     print(selected_columns)
    if selected_columns is None:
        return []
    
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# This callback will update the geo-location chart for the selected data entry               
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    print('index: ' + str(index))
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can 
    # be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    print('row: ' + str(row))
    
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
            center=[dff.iloc[row,12],dff.iloc[row,13]], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 12 and 13 define the grid-coordinates for 
            # the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,12],dff.iloc[row,13]],
                children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]

# Open in new tab:
app.run_server(port=18843)